In [94]:
from pyspark.sql import SparkSession
import pandas as pd

# initialise sparkContext

spark = SparkSession.builder \
    .master('local') \
    .appName('myAppName') \
    .config('spark.executor.memory', '5gb') \
    .config("spark.cores.max", "6") \
    .getOrCreate()

sc = spark.sparkContext

# using SQLContext to read parquet file

from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

# to read parquet files

df_immi = sqlContext.read.parquet("tables/immigration_fact/part-00000-8da82395-49d1-4b95-8578-82f74ab7e309-c000.snappy.parquet")
df_country = sqlContext.read.parquet("tables/country")
df_demographics = sqlContext.read.parquet("tables/demographics")

## Top 10 Cities with Highest Percentage of Foreign Born Population

In [95]:
from pyspark.sql.functions import col
from pyspark.sql.functions import desc


df_demographics2=df_demographics.withColumn("Percentage", col("foreign_born")/col("total_population"))
#df_demographics2.filter(col("Percentage") == "1")
#df_demographics2.select(col("State"),col("City"),col("Percentage")).sort(desc("Percentage")).limit(10).filter(col("Percentage") > "0.05").distinct().limit(5).toPandas()
df_demographics2.select(col("State"),col("state_Code"),col("City"),col("Percentage")).sort(desc("Percentage")).distinct().limit(10).toPandas()


,State,state_Code,City,Percentage
0,Florida,FL,Hialeah,0.717676
1,Florida,FL,Miami,0.591373
2,New Jersey,NJ,Union City,0.586458
3,California,CA,Glendale,0.554707
4,Florida,FL,Miami Beach,0.540651
5,California,CA,Daly City,0.531462
6,California,CA,Alhambra,0.519340
7,California,CA,El Monte,0.510985
8,California,CA,Milpitas,0.508085
9,New Jersey,NJ,Elizabeth,0.491528


## Arrivals more than 50000

In [104]:
#df_demographics2=df_demographics.withColumn("Percentage", col("foreign_born")/col("total_population"))
#df_demographics2.filter(col("Percentage") == "1")
#df_demographics2.select(col("State"),col("City"),col("Percentage")).sort(desc("Percentage")).limit(10).filter(col("Percentage") > "0.05").distinct().limit(5).toPandas()
df_immi.groupBy("state_code", "country_residence_code").count().where(col("count")>50000).toPandas()

,state_code,country_residence_code,count
0,NY,135.0,75503
1,NY,111.0,80327
2,CA,245.0,55863
3,FL,689.0,64572
4,FL,135.0,101913
5,HI,209.0,97768


## Countries from where the arrivals are more than 50000 to different US states

In [137]:
#create dataframe to join with country df so that we dont have to scan the whole immigration table
df_immi2 =df_immi.groupBy("state_code", "country_residence_code").count()
df_country_immi = df_country.join(df_immi2,df_country.country_code ==  df_immi2.country_residence_code,"inner")
#display(df_country_immi)
df_country_immi.select(col("country_name"),col("state_Code"), col("count")).where (col("count")>50000).toPandas()


,country_name,state_Code,count
0,United Kingdom,NY,75503
1,France,NY,80327
2,"China, Prc",CA,55863
3,Brazil,FL,64572
4,United Kingdom,FL,101913
5,Japan,HI,97768
